# (a)

In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
#loading What's Cooking data into the file 
train_data_Cooking = pd.read_json('train.json')
test_data_Cooking = pd.read_json('test.json')
train_data_Cooking.head(5)

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


# (b)

In [3]:
#find the number of samples in the training set 
train_data_size = train_data_Cooking.shape
#find the number of type of cuisines in the training set 
type_of_cuisine = train_data_Cooking.cuisine.nunique()
#create a list that keeps all the ingredients
all_ingredients = np.array(train_data_Cooking["ingredients"].values)
#create a list that has all the unique ingredients
unique_ingredients = np.unique(np.hstack(all_ingredients))
#calculates the number of unique ingredients 
num_unique_ingredients = len(unique_ingredients)
print("There are", type_of_cuisine, "unique cuisines and", num_unique_ingredients, "unique ingredients")

There are 20 unique cuisines and 6714 unique ingredients


### There are 39774 dishes in the training set, 20 types of cuisines and 6714 unique ingredients.

# (c)

In [4]:
train_label = train_data_Cooking['cuisine'].values
train_data = train_data_Cooking.drop(['cuisine','id'], axis = 1)
test_id = test_data_Cooking['id'].values
test_data = test_data_Cooking.drop(['id'], axis = 1)

In [5]:
# Label encoder to convert the labels into vectors
from sklearn.preprocessing import LabelEncoder
# Convert the data into vectors
from sklearn.preprocessing import MultiLabelBinarizer

In [6]:
#use one hot encoder to transform categorical variables to binary feature vectors
mlb = MultiLabelBinarizer()
train_data_clean = train_data.join(pd.DataFrame(mlb.fit_transform(train_data.pop('ingredients')),columns = mlb.classes_, index = train_data.index))
train_data_clean.head(5)

,( oz.) tomato sauce,( oz.) tomato paste,(10 oz.) frozen chopped spinach,"(10 oz.) frozen chopped spinach, thawed and squeezed dry",(14 oz.) sweetened condensed milk,(14.5 oz.) diced tomatoes,(15 oz.) refried beans,1% low-fat buttermilk,1% low-fat chocolate milk,1% low-fat cottage cheese,...,yukon gold potatoes,yuzu,yuzu juice,za'atar,zest,zesty italian dressing,zinfandel,ziti,zucchini,zucchini blossoms
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Transform the test data too
mlb_2 = MultiLabelBinarizer()
test_data_clean = test_data.join(pd.DataFrame(mlb.transform(test_data.pop('ingredients')),columns = mlb.classes_, index = test_data.index))

/Users/hangyulin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['2% reduced fat chocolate milk', '33% less sodium smoked ham', '50% less sodium black beans', 'Asian herb', 'Best Foods Mayonnaise Dressing with Extra Virgin Olive Oil', 'Boursin Cheese with Garlic and Herbs', 'Bramley apples', 'CURRY GUY Smoked Tandoori Masala', 'Chobani Yogurt', 'Crisco Canola Oil', 'Daisy Brand Light Sour Cream', 'Del Monte Diced Tomatoes', 'Domaine de Canton Ginger Liqueur', 'Dreamfields Lasagna', 'Dubliner cheese', 'Duncan Hines Classic White Cake Mix', 'Foster Farms chicken drumsticks', 'Franks Wings Sauce', "French's Spicy Brown Mustard", 'Goya Corn Oil', 'Goya Seasoning', 'Green Giant™ Steamers™ Niblets® frozen corn', 'Green Giant™ frozen chopped spinach', 'Grey Poupon Dijon Mustard', "Hellmann's Mayonnaise with a hint of Wasabi", 'Hershey bars', 'Holland House White Cooking Wine', 'Honey Bunches of Oats Cereal', 'Johnsonville® Italian All 

In [8]:
# Convert the label too
label_enc = LabelEncoder()
train_label_vec = label_enc.fit_transform(train_label)

In [9]:
# Convert into numpy array
train_data_vec = train_data_clean.values
test_data_vec = test_data_clean.values

## (d)

In [10]:
# Use pipeline to run model
from sklearn.pipeline import Pipeline
# Using gridsearch to search for best hyperparameters
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

### Gaussian distribution Naive Bayes Classifier

In [19]:
# Use Gaussian distribution prior assumption for Naive Bayes Classifier
grid_param_gnb = [{}]

gnb = GaussianNB()

grid_search_gnb = GridSearchCV(gnb, grid_param_gnb, cv = 3,
                           scoring='accuracy', return_train_score=True)

grid_search_gnb.fit(train_data_vec, train_label_vec)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=GaussianNB(priors=None, var_smoothing=1e-09),
       fit_params=None, iid='warn', n_jobs=None, param_grid=[{}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [23]:
print("The accuracy for Gaussian distribution prior assumption Naïve Bayes Classifier is", round(grid_search_gnb.best_score_,6))

The accuracy for Gaussian distribution prior assumption Naïve Bayes Classifier is 0.379494


### Bernoulli distribution Naive Bayes Classifier

In [32]:
# Use Bernoulli distribution prior assumption for Naive Bayes Classifier
grid_param_bnb = [{'alpha': np.arange(0.1, 1, 0.1)}]

bnb = BernoulliNB()

grid_search_bnb = GridSearchCV(bnb, grid_param_bnb, cv = 3,
                           scoring='accuracy', return_train_score=True)

grid_search_bnb.fit(train_data_vec, train_label_vec)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [35]:
print("The accuracy for Bernoulli distribution prior assumption Naïve Bayes Classifier is", round(grid_search_bnb.best_score_,6),
      "with best alpha of", grid_search_bnb.best_params_)

The accuracy for Bernoulli distribution prior assumption Naïve Bayes Classifier is 0.744959 with best alpha of {'alpha': 0.1}


# e)

Gaussian Accuracy is 
Bernoulli Accuracy is 
From 3-fold cross validation,  performs better 

# f)

In [11]:
# Importing logistic regression model from sklearn 
from sklearn.linear_model import LogisticRegression

In [49]:
# Use Gaussian distribution prior assumption for Naive Bayes Classifier
grid_param_lg = [{"C": np.arange(0.5, 2, 0.25)}]

lg = LogisticRegression(solver='lbfgs', multi_class='auto', penalty = "l2", max_iter = 4000)

grid_search_lg = GridSearchCV(lg, grid_param_lg, cv = 3,
                           scoring='accuracy', return_train_score=True)

grid_search_lg.fit(train_data_vec, train_label_vec)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=4000, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'C': array([0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [50]:
print("The accuracy for Bernoulli distribution prior assumption Naïve Bayes Classifier is", round(grid_search_lg.best_score_,6),
      "with best alpha of", grid_search_lg.best_params_)

The accuracy for Bernoulli distribution prior assumption Naïve Bayes Classifier is 0.772917 with best alpha of {'C': 0.75}


# g)
The best model was the logistic regression model

In [94]:
# Use linear regression to predict the test data
test_pred = grid_search_lg.best_estimator_.predict(test_data_vec)
test_pred_label = label_enc.inverse_transform(test_pred)

In [96]:
test_pd = pd.DataFrame({"id":test_id, "cuisine":test_pred_label})
test_pd.head()

,id,cuisine
0,18009,british
1,28583,southern_us
2,41580,italian
3,29752,cajun_creole
4,35687,italian


In [100]:
test_pd.to_csv("Question2_submission.csv", index = False)

In [12]:
from sklearn.decomposition import PCA
pca = PCA()
lgr = LogisticRegression(solver='lbfgs', multi_class='auto', penalty = "l2", max_iter = 4000)

In [13]:
model_pipe = Pipeline(steps=[("PCA", pca),("LGR", lgr)])

In [30]:
param_grid = {
    'PCA__n_components': [1500],
    'LGR__C': np.arange(0.7, 0.9, 0.1)
}

In [31]:
super_grid = GridSearchCV(model_pipe, param_grid, cv = 3, n_jobs=-1)

In [32]:
super_grid.fit(train_data_vec, train_label_vec)

/Users/hangyulin/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: 

In [ ]:
super_grid.best_score_

In [ ]:
super_grid.best_params_